In [33]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.metrics import  make_scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score, silhouette_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import joblib

In [2]:
!where python

c:\Users\polyx\Desktop\Pattern-Recognition-Gr7\venv\Scripts\python.exe
C:\Users\polyx\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\polyx\AppData\Local\Microsoft\WindowsApps\python.exe


In [17]:
import sklearn
print(np.__version__)

2.1.2


In [40]:
with open('No_cv_grid_search.pkl', 'rb') as file:
    grid_search_results = pickle.load(file)

print(grid_search_results.cv_results_)

{'mean_fit_time': array([8.06002507e+01, 8.09346490e+01, 8.14747469e+01, 8.18005538e+01,
       8.05469654e+01, 8.14188638e+01, 8.17702982e+01, 8.64599297e+01,
       8.04620423e+01, 8.25756171e+01, 8.28455067e+01, 8.58129725e+01,
       8.15524418e+01, 8.09954488e+01, 8.14131174e+01, 8.30446229e+01,
       8.08539884e+01, 8.11842804e+01, 8.19340742e+01, 8.38439443e+01,
       8.14197199e+01, 8.24718990e+01, 8.32894876e+01, 8.55833552e+01,
       8.11516759e+01, 8.12417147e+01, 8.15023091e+01, 8.26847022e+01,
       8.10909562e+01, 8.21635170e+01, 8.26179297e+01, 8.25349305e+01,
       8.41219752e+01, 8.19988875e+01, 8.87182920e+01, 8.50212433e+01,
       8.05357065e+01, 7.94278781e+01, 8.28028188e+01, 8.12009938e+01,
       8.02740915e+01, 8.12896931e+01, 8.12110465e+01, 8.21458762e+01,
       8.02455673e+01, 8.00583436e+01, 7.94143612e+01, 8.07383413e+01,
       8.12319875e+01, 8.48328927e+01, 7.84462690e+01, 7.87515697e+01,
       7.98110919e+01, 8.02247045e+01, 7.86931605e+01, 7.93

In [41]:
def create_ranked_lists(grid_search_results):

    cv_results = grid_search_results.cv_results_
    
    df = pd.DataFrame({
        'mean_fit_time': cv_results['mean_fit_time'],
        'mean_score_time': cv_results['mean_score_time'],
        'mean_test_f1': cv_results['mean_test_f1'],
        'params': cv_results['params']  # Access to model configurations
    })

    # Extract F1 scores
    f1_scores = cv_results.get('mean_test_f1', None)
    
    if f1_scores is not None:
        # Count how many have a perfect F1 score of 1.0
        perfect_f1_count = sum(1 for score in f1_scores if score == 1.0)
        print(f"Number of models with a perfect F1 score: {perfect_f1_count}")


        # Filter to include only models with a perfect F1 score
    df_perfect_f1 = df[df['mean_test_f1'] == 1.0]

    # Rank models based on mean_score_time
    df_ranked_by_score_time = df_perfect_f1.sort_values(by='mean_score_time', ascending=True).reset_index(drop=True)

    # Rank models based on mean_fit_time
    df_ranked_by_fit_time = df_perfect_f1.sort_values(by='mean_fit_time', ascending=True).reset_index(drop=True)

    return df_ranked_by_score_time, df_ranked_by_fit_time

In [42]:
ranked_by_score_time, ranked_by_fit_time = create_ranked_lists(grid_search_results)

print("Ranked by Mean Score Time:")
print(ranked_by_score_time[['mean_score_time', 'mean_test_f1', 'params']])
print("\nRanked by Mean Fit Time:")
print(ranked_by_fit_time[['mean_fit_time', 'mean_test_f1', 'params']])

Number of models with a perfect F1 score: 131
Ranked by Mean Score Time:
     mean_score_time  mean_test_f1  \
0           0.005817           1.0   
1           0.005997           1.0   
2           0.007086           1.0   
3           0.007511           1.0   
4           0.008195           1.0   
..               ...           ...   
126         0.196272           1.0   
127         0.214792           1.0   
128         0.218323           1.0   
129         0.294248           1.0   
130         0.427850           1.0   

                                                params  
0    {'classifier': SVC(probability=True, random_st...  
1    {'classifier': SVC(probability=True, random_st...  
2    {'classifier': SVC(probability=True, random_st...  
3    {'classifier': SVC(probability=True, random_st...  
4    {'classifier': KNeighborsClassifier(), 'classi...  
..                                                 ...  
126  {'classifier': KNeighborsClassifier(), 'classi...  
127  {'classif

In [21]:
ranked_by_score_time[['params'][0]][0]

{'classifier': RandomForestClassifier(random_state=12),
 'feature_selection': SelectKBest(score_func=<function mutual_info_classif at 0x00000289BB349090>),
 'feature_selection__k': 500}